In [1]:
import ee
ee.Initialize()

In [2]:
import geepy
import math
import sys

In [3]:
config = geepy.params.configParams('input_classification_v31.json')

bands = config.params['bandParams']

l5 = ee.ImageCollection(config.params['imgCollection']['lc5']['id']).select(
    config.params["imgCollection"]["lc5"]["bands"],
    config.params["imgCollection"]["lc5"]["bandNames"])

l7 = ee.ImageCollection(config.params['imgCollection']['lc7']['id']).select(
    config.params["imgCollection"]["lc7"]["bands"],
    config.params["imgCollection"]["lc7"]["bandNames"])

l8 = ee.ImageCollection(config.params['imgCollection']['lc8']['id']).select(
    config.params["imgCollection"]["lc8"]["bands"],
    config.params["imgCollection"]["lc8"]["bandNames"])

In [4]:
#watersheds = ee.FeatureCollection(config.params['studyArea'])

In [5]:
tiles = ee.FeatureCollection(config.params['lsTiles'])

In [6]:
tile = geepy.feature.getTiles(tiles, 'T'+config.params['grid']['tiles'][7], 'TILE_T')
tile = tile[1]

In [7]:
landsat = {}
landsat_min = {}
landsat_max = {}

for year in config.params['years2process']:  
    start_d = year + config.params['period']['dry']['start']
    end_d = year + config.params['period']['dry']['end']


    #print("Start-End dry season %s %s" %(start_d, end_d))

    if(int(year) < 2002):
        filtered = l5.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d).map(geepy.image.maskLandsatSR)
        satellite = 'l5'
    elif(int(year) in (2002, 2011, 2012)):
        filtered = l7.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d).map(geepy.image.maskLandsatSR)
        satellite = 'l7'
    elif(int(year) > 2002 and int(year) < 2011):
        filtered = l5.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d).map(geepy.image.maskLandsatSR)
        satellite = 'l5'
    elif(int(year) == 2018):
        filtered = l8.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, '2018-09-06').map(geepy.image.maskLandsatSR)
        satellite = 'l8'
    else:
        filtered = l8.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d).map(geepy.image.maskLandsatSR)
        satellite = 'l8'
    
    fEdgeRemoved = filtered.map(geepy.image.edgeRemoval).median()
    fEdgeRemoved_min = filtered.map(geepy.image.edgeRemoval).min()
    fEdgeRemoved_max = filtered.map(geepy.image.edgeRemoval).max()
    
    landsat[year] = fEdgeRemoved.clip(tile)
    
    landsat_min[year] = fEdgeRemoved_min.clip(tile)
    landsat_max[year] = fEdgeRemoved_max.clip(tile)
    
    sys.stdout.write("\rProcessing Landsat data: %s" % year)
    sys.stdout.flush()

Processing Landsat data: 2018

In [8]:
def normalize(img, bandMin, bandMax, bandName):
    
    numer = img.select(bandName).subtract(bandMin)
    denom = bandMax - bandMin
    
    res = numer.divide(denom)
    
    return img.addBands(res.rename(['norm'+bandName]))

In [10]:
minMax = fEdgeRemoved_max.reduceRegion(
    reducer = ee.Reducer.minMax(),
    scale = 30,
    geometry = tile,
    maxPixels = 1e13,
    bestEffort = True
)

In [11]:
print(ee.Number(minMax.get('red_max')).getInfo())
print(ee.Number(minMax.get('green_max')).getInfo())
print(ee.Number(minMax.get('blue_max')).getInfo())
print(ee.Number(minMax.get('nir_max')).getInfo())
print(ee.Number(minMax.get('swir1_max')).getInfo())
print(ee.Number(minMax.get('swir2_max')).getInfo())

0.90420001745224
0.7091000080108643
0.6309000253677368
1.373900055885315
1.600000023841858
1.600000023841858


In [ ]:
for year in config.params['years2process']:
    landsat[year] = normalize(landsat[str(year)],
              -0.200, 
              1.600, 'red')

In [ ]:
print(ee.Number(minMax.get('red_min')).getInfo())
print(ee.Number(minMax.get('green_min')).getInfo())
print(ee.Number(minMax.get('blue_min')).getInfo())
print(ee.Number(minMax.get('nir_min')).getInfo())
print(ee.Number(minMax.get('swir1_min')).getInfo())
print(ee.Number(minMax.get('swir2_min')).getInfo())

In [ ]:
mapa = geepy.maps.geeMap(tile, zoom=10)
#mapa.addLayer(landsat['1990'], viz_params={'min':0.15,'max':0.4,'bands':'swir1,nir,red'}, name=year)

mapa.addLayer(landsat['1990'], viz_params={'min':0.15,'max':0.4,'bands':'normswir1,normnir,normred'}, name='norm1990')
#mapa.addLayer(landsat['2000'], viz_params={'min':0.15,'max':0.4,'bands':'normswir1,normnir,normred'}, name='norm2000')
#mapa.addLayer(landsat['2018'], viz_params={'min':0.15,'max':0.4,'bands':'swir1,nir,red'}, name='norm2014')
#mapa.addLayer(landsat['1990'], viz_params={'min':0.15,'max':0.4,'bands':'swir1_min,nir_min,red_min'}, name='norm2018')
mapa.addControls()

mapa.show()

In [ ]:
task = geepy.image.send2drive(landsat['2018'].select(['normswir1', 'normnir', 'normred']), coords, 'normalization', 'normal', 30)

In [ ]:
task.start()